In [1]:
import numpy as np
import pandas as pd

/tmp/ipykernel_14714/1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
gamma = 0.9
alpha = 0.6
door = -20
wall = -200
home = 1000

In [3]:
rewardsTable = np.array([
    [0,-1,0,0,wall,0,0,0,0,0,0,0,0,0,0,0],
    [-1,0,-1,0,0,door,0,0,0,0,0,0,0,0,0,0],
    [0,-1,0,-1,0,0,wall,0,0,0,0,0,0,0,0,0],
    [0,0,-1,0,0,0,0,-1,0,0,0,0,0,0,0,0],
    [-1,0,0,0,0,door,0,0,-1,0,0,0,0,0,0,0],
    [0,-1,0,0,wall,0,wall,0,0,-1,0,0,0,0,0,0],
    [0,0,-1,0,0,door,0,-1,0,0,-1,0,0,0,0,0,0],
    [0,0,0,-1,0,0,wall,0,0,0,0,-1,0,0,0,0],
    [0,0,0,0,wall,0,0,0,0,-1,0,0,-1,0,0,0],
    [0,0,0,0,0,door,0,0,-1,0,-1,0,0,home,0,0],
    [0,0,0,0,0,0,wall,0,0,-1,0,-1,0,0,-1,0],
    [0,0,0,0,0,0,0,-1,0,0,-1,0,0,0,0,-1],
    [0,0,0,0,0,0,0,0,-1,0,0,0,0,home,0,0],
    [0,0,0,0,0,0,0,0,0,-1,0,0,-1,0,-1,0],
    [0,0,0,0,0,0,0,0,0,0,-1,0,0,home,0,-1],
    [0,0,0,0,0,0,0,0,0,0,0,-1,0,0,-1,0]
    ])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (16,) + inhomogeneous part.

In [ ]:
coordinatesToStates = {
    'A1' : 0,
    'B1' : 1,
    'C1' : 2,
    'D1' : 3,
    'A2' : 4,
    'B2' : 5,
    'C2' : 6,
    'D2' : 7,
    'A3' : 8,
    'B3' : 9,
    'C3' : 10,
    'D3' : 11,
    'A4' : 12,
    'B4' : 13,
    'C4' : 14,
    'D4' : 15,
}


In [ ]:
possibleActions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]


In [ ]:
stateToCoordinates = {state: coordinates for coordinates, state in coordinatesToStates.items()}

def policy(stateT, legalActions, QValues, method, epsilon):
    if method == 'QLearning':
        return np.random.choice(legalActions)
    if method == 'SARSA':
        if np.random.random() < epsilon:
            return np.random.choice(legalActions)
        else:
            bestAction = legalActions[0]
            for index in legalActions:
                if QValues[stateT][index] > bestAction:
                    bestAction = index
            return bestAction

def legalActionChecker(index = 15, rewards, state):
    legalActions = []
        while i <= index:
            if rewards[state][i] != 0:
                legalActions.append(i)
            i+=1
    return legalActions

def path(QValues, startCoord = 'A1', endCoord = 'B4'):
    path = [startCoord]
    reward = 0
    nextCoord = startCoord
    while (nextCoord != endCoord):
        startState = coordinatesToStates[startCoord]

        nextState = np.argmax(QValues[startState,])
        nextCoord = stateToCoordinates[nextState]
        reward += rewardsTable[startState][nextState]
        path.append(nextCoord)
        startCoord = nextCoord
    return {'Reward': reward, 'Path':path}

def qLearning(episodeCount, method, epsilon):
    currentEpisode = 0
    QValues = np.array(np.zeros([16,16]))

    while currentEpisode <= episodeCount:
        stateT = 0
        stateNext = int
        legalActions = []
        reward = 0
        i = 0

        #While haven't reached the terminal state
        while stateNext != 13:
            legalActions = legalActionChecker(15,rewardsTable,stateT)
            #stateNext = np.random.choice(legalActions)
            stateNext= policy(stateT, legalActions,QValues,method,epsilon)
            reward += rewardsTable[stateT][stateNext]
            TD = rewardsTable[stateT][stateNext] + gamma * QValues[stateNext][np.argmax(QValues[stateNext])] - QValues[stateT][stateNext]
            QValues[stateT][stateNext] += alpha*TD 
            stateT=stateNext
            legalActions = []
            i = 0
        #print(QValues.astype(int))
        print(path(QValues))
        currentEpisode += 1
    return QValues

In [ ]:
stateToCoordinates = {state: coordinates for coordinates, state in coordinatesToStates.items()}

In [ ]:
pd.DataFrame(qLearning(1,'QLearning', 0.5).astype(int), columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

#print(qLearning(15,'QLearning', 0.5).astype(int))
#print(qLearning(15,'SARSA', 0.5).astype(int))
#print(path())

In [ ]:
pd.DataFrame(qLearning(10,'SARSA', 0.5).astype(int), columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])


In [ ]:
print(qLearning(20000,'QLearning', 0.5).astype(int))